In [14]:
import pandas as pd
import numpy as np

In [15]:
########################    Compute the odds-ratio of every race to smoking    ##################

In [16]:
data = pd.read_csv('../data/nc-est2019-alldata-r-file20.csv')

In [17]:
data = data[data['UNIVERSE']=='R']
data = data[data['MONTH']==7]
data = data[data['AGE']>=18]
data = data[data['AGE']!=999]
data

,UNIVERSE,MONTH,YEAR,AGE,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,BA_MALE,...,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
19,R,7,2019,18,4255827,2172385,2083442,1594641,1519747,326121,...,463823,445782,34523,32775,22009,21324,10531,9955,3442,3471
20,R,7,2019,19,4330439,2214784,2115655,1624910,1541428,336758,...,466593,444897,34296,33044,22464,21454,10231,9914,3406,3322
21,R,7,2019,20,4269683,2183139,2086544,1605868,1524953,329229,...,452703,432562,33209,31718,21498,20786,10116,9818,3441,3158
22,R,7,2019,21,4278323,2187086,2091237,1605637,1524445,331443,...,448778,426900,32289,30964,21565,20386,9732,9422,3233,3098
23,R,7,2019,22,4298772,2195296,2103476,1610257,1532651,330789,...,446546,424333,32065,30378,21613,20380,9680,9284,3269,3042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,R,7,2019,96,157463,46046,111417,39789,96090,2947,...,3242,6635,119,282,83,137,31,64,15,19
98,R,7,2019,97,116969,32611,84358,27974,72835,2083,...,2132,4888,78,220,49,109,27,53,9,20
99,R,7,2019,98,86150,23543,62607,20241,53884,1474,...,1431,3434,58,149,40,88,15,38,8,13
100,R,7,2019,99,57124,14596,42528,12290,35971,1003,...,938,2378,41,111,27,50,18,23,7,13


In [18]:
cols = ['White, non-Hispanic', 'Black, non-Hispanic', 'Asian, non-Hispanic',
       'American Indian/Alaska Native, non-Hispanic', 'Hispanic', 'Other, non-Hispanic']
table_race = pd.DataFrame(columns=cols)

smoking_percents = [15.5, 14.9, 7.2, 20.9, 8.8, 19.7]
inits = ['NHWA_', 'NHBA_', 'NHAA_', 'NHIA_', 'H_', 'NHNA_']

for col,smok_per,init in zip(cols,smoking_percents,inits):
    pop_size = data.loc[:,[init+g for g in ['MALE', 'FEMALE']]].sum().sum()
    table_race.loc['population_size', col] = pop_size
    table_race.loc['smoking', col] = smok_per/100*pop_size
    table_race.loc['non-smoking', col] = (100-smok_per)/100*pop_size

#switch to numbers in thousands
table_race = table_race/1000
table_race

,"White, non-Hispanic","Black, non-Hispanic","Asian, non-Hispanic","American Indian/Alaska Native, non-Hispanic",Hispanic,"Other, non-Hispanic"
population_size,160627,31140.3,15221.8,1818.96,41884.7,448.851
smoking,24897.2,4639.91,1095.97,380.162,3685.85,88.4236
non-smoking,135730,26500.4,14125.8,1438.8,38198.8,360.427


In [19]:
# Compute odds-ratio
for col in cols:
    table_race.loc['OR', col] = ((table_race.loc['smoking', col]/(table_race.loc['smoking', [c for c in cols if c!=col]].sum()))/
                            (table_race.loc['non-smoking', col]/(table_race.loc['non-smoking', [c for c in cols if c!=col]].sum())))

table_race

,"White, non-Hispanic","Black, non-Hispanic","Asian, non-Hispanic","American Indian/Alaska Native, non-Hispanic",Hispanic,"Other, non-Hispanic"
population_size,160627,31140.3,15221.8,1818.96,41884.7,448.851
smoking,24897.2,4639.91,1095.97,380.162,3685.85,88.4236
non-smoking,135730,26500.4,14125.8,1438.8,38198.8,360.427
OR,1.49531,1.10261,0.465699,1.65039,0.552717,1.52712


In [20]:
########################    Compute the odds-ratio of socioeconomic status to smoking    ##################

In [21]:
cols = ['<35,000','35,000–74,999','75,000–99,999','≥100,000']
smoking_percents = [21.4, 15.7, 11.4, 7.1]

#number_housholds = 128451
#percent_of_total_population = [25.4, 28.2, 12.3, 34.1]

# Load data of number of people (in thousands) vs. household income
data = pd.read_excel('../data/hinc03_1_1.xlsx', engine='openpyxl')
data = data.iloc[8:]
data.columns = data.iloc[0,:]
data = data.iloc[1:,:]
data.columns = ['income_range']+['male_'+x for x in list(data)[1:7]]+['female_'+x for x in list(data)[7:]]
data.index = range(len(data))
#data

In [22]:
number_people_in_households = []
number_people_in_households.append(data.loc[1:7,['male_Total','female_Total']].sum().sum())
number_people_in_households.append(data.loc[8:15,['male_Total','female_Total']].sum().sum())
number_people_in_households.append(data.loc[16:20,['male_Total','female_Total']].sum().sum())
number_people_in_households.append(data.loc[21:41,['male_Total','female_Total']].sum().sum())

number_people_in_households

[59869.0, 86591.0, 42862.0, 135736.0]

In [23]:
# compute the number of children (under 18) per household income and reduce it from the total number of people
files = ['../data/hinc03_2_1.xlsx',
        '../data/hinc03_3_1.xlsx',
        '../data/hinc03_4_1.xlsx',
        '../data/hinc03_5_1.xlsx']

number_children_in_households = np.zeros((4))

for f,file in enumerate(files):
    data1 = pd.read_excel(file, engine='openpyxl')
    data1 = data1.iloc[8:]
    data1.columns = data1.iloc[0,:]
    data1 = data1.iloc[1:,:]
    data1.columns = ['income_range']+['male_'+x for x in list(data1)[1:7]]+['female_'+x for x in list(data1)[7:]]
    data1.index = range(len(data1))
    number_children_in_households[0] += data1.loc[1:7,['male_Total','female_Total']].sum().sum()
    number_children_in_households[1] += data1.loc[8:15,['male_Total','female_Total']].sum().sum()
    number_children_in_households[2] += data1.loc[16:20,['male_Total','female_Total']].sum().sum()
    number_children_in_households[3] += data1.loc[21:41,['male_Total','female_Total']].sum().sum()
    
number_adults_in_households = list(np.array(number_people_in_households)-number_children_in_households)
number_adults_in_households

[46988.0, 67363.0, 33448.0, 104157.0]

In [24]:
table_income = pd.DataFrame(columns=cols)
for col,smok_per,num_people in zip(cols,smoking_percents,number_adults_in_households):
    table_income.loc['population_size', col] = num_people
    table_income.loc['smoking', col] = smok_per/100*num_people
    table_income.loc['non-smoking', col] = (100-smok_per)/100*num_people

table_income

,"<35,000","35,000–74,999","75,000–99,999","≥100,000"
population_size,46988,67363,33448,104157
smoking,10055.4,10576,3813.07,7395.15
non-smoking,36932.6,56787,29634.9,96761.9


In [25]:
# Compute odds-ratio
for col in cols:
    table_income.loc['OR', col] = ((table_income.loc['smoking', col]/(table_income.loc['smoking', [c for c in cols if c!=col]].sum()))/
                            (table_income.loc['non-smoking', col]/(table_income.loc['non-smoking', [c for c in cols if c!=col]].sum())))

table_income

,"<35,000","35,000–74,999","75,000–99,999","≥100,000"
population_size,46988,67363,33448,104157
smoking,10055.4,10576,3813.07,7395.15
non-smoking,36932.6,56787,29634.9,96761.9
OR,2.28948,1.43053,0.874488,0.385671
